In [1]:
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")

# model = AutoModelForMaskedLM.from_pretrained("GroNLP/bert-base-dutch-cased")

In [1]:
import os
from lxml import etree

path = 'C14NL-PoS/C14NL-PoS'
    
files = os.listdir(path)

def get_flattext(tree):
    body = []
    for w in tree.iterfind('.//w'):
        body.append(''.join(w.itertext()).strip().replace('\n               ', ''))
    flattext = ' '.join(body)
    return flattext

def get_text_with_ents(tree):
    body_with_ents = []
    for w in tree.iterfind('.//w'):
        try:
            if w.attrib['ana'].find('c02') != -1:
                txt_ent = ''.join(w.itertext()).strip()
                txt_ent = '{' + txt_ent + '}'
                body_with_ents.append(txt_ent)
            else:
                body_with_ents.append(''.join(w.itertext()).strip().replace('\n               ', ''))
        except KeyError:
            body_with_ents.append(''.join(w.itertext()).strip().replace('\n               ', ''))
    return ' '.join(body_with_ents)

def get_text_with_ents_list(tree):
    body_with_ents = []
    for w in tree.iterfind('.//w'):
        try:
            if w.attrib['ana'].find('c02') != -1:
                txt_ent = ''.join(w.itertext()).strip()
                for e in txt_ent.split():
                    body_with_ents.append((e, 'e'))
            else:
                body_with_ents.append(''.join(w.itertext()).strip().replace('\n               ', ''))
        except KeyError:
            body_with_ents.append(''.join(w.itertext()).strip().replace('\n               ', ''))
    return body_with_ents

def get_ents_list(tree):
    ents = []
    for w in tree.iterfind('.//w'):
        try:
            if w.attrib['ana'].find('c02') != -1:
                txt_ent = ''.join(w.itertext()) 
                ents.append(txt_ent.strip().replace('\n               ', ''))
        except KeyError:
            continue
    return ents

def print_xml(element):
    "functie om xml mooi te printen"
    print(etree.tostring(element, pretty_print=True).decode('utf-8'))

with open(path + "/" + files[5]) as f:
    tree = etree.parse(f)

FileNotFoundError: [WinError 3] Het systeem kan het opgegeven pad niet vinden: 'C14NL-PoS/C14NL-PoS'

In [2]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
# do_download = False
# if do_download:
#     for dl in download("dum"):
#         x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "dum"
tagger = get_tagger(model_name, batch_size=256, device="cpu")

sentences: List[str] = [get_flattext(tree)]
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.dum.imports import get_iterator_and_processor
for sentence_group in sentences:
    iterator, processor = get_iterator_and_processor()
    thing = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
    print(thing)

2022-03-22 18:08:29,208 : Model C:\Users\Gebruiker\anaconda3\lib\site-packages\pie_extended\downloads\dum\lemma.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.
2022-03-22 18:08:29,916 : Model C:\Users\Gebruiker\anaconda3\lib\site-packages\pie_extended\downloads\dum\pos.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.


[{'form': 'WiE', 'lemma': 'het', 'pos': 'pron(pers,1,sing)'}, {'form': 'pauwels', 'lemma': 'paulus', 'pos': 'n(prop)'}, {'form': 'vlaminc', 'lemma': 'vlaming', 'pos': 'n(prop)'}, {'form': 'mich', 'lemma': 'ik', 'pos': 'pron(pers,1,sing)'}, {'form': 'de', 'lemma': 'de', 'pos': 'art(def)'}, {'form': 'ghier', 'lemma': 'gier', 'pos': 'n(sing)'}, {'form': 'fense', 'lemma': 'fens', 'pos': 'n(prop)'}, {'form': 'ghoedybeur', 'lemma': 'goedgebuur', 'pos': 'n(prop)'}, {'form': 'jan', 'lemma': 'johannes', 'pos': 'n(prop)'}, {'form': 'de', 'lemma': 'de', 'pos': 'art(def)'}, {'form': 'wale', 'lemma': 'waal', 'pos': 'n(prop)'}, {'form': 'en', 'lemma': 'en', 'pos': 'conj(coord)'}, {'form': 'clais', 'lemma': 'nicolaas', 'pos': 'n(prop)'}, {'form': 'de', 'lemma': 'de', 'pos': 'art(def)'}, {'form': 'hane', 'lemma': 'haan', 'pos': 'n(prop)'}, {'form': 'scepene', 'lemma': 'schepen', 'pos': 'n(plu)'}, {'form': 'mire', 'lemma': 'mijn', 'pos': 'pron(poss)'}, {'form': 'joncvrowe', 'lemma': 'jonkvrouw', 'pos':

In [3]:
# print(thing)
lst = []

# print(thing)

for dct in thing:
#     print(f"lemma = {dct['lemma']}")
    if dct['pos'] == 'n(prop)':
        ent = '{'+ dct['form'] + '}'
        lst.append(ent)
    else:
        lst.append(dct['form'])

txt_with_ents_pie = ' '.join(lst)
txt_with_ents_true = get_text_with_ents(tree)

print(txt_with_ents_pie)
print(txt_with_ents_true)

ents = []
for dct in thing:
#     print(f"lemma = {dct['lemma']}")
    if dct['pos'] == 'n(prop)':
        ent = dct['form']
        ents.append(ent)

ents2 = get_ents_list(tree)

WiE {pauwels} {vlaminc} mich de ghier {fense} {ghoedybeur} {jan} de {wale} en {clais} de {hane} scepene mire joncvrowe van {pasquedale} in de prochhie van {langhemarc} doen te weitene alle deghone die deise lette {zulle} zien of hore leisen dat {jacob} {gherboud} porte van {yppre} heift ghecocht en ghecreghe Eruelike wel en wettelike te zine bouf en te sijns hoirs bouf jeghe {janne} de amman ser {jans} zone en ieghe coste zin wijf jeghen {clais} {boitac} labrechtszone en ieghe {lisebette} zin wijf en ieghe piet {keyen} {willems} zone en ieghe {jakemine} zin wijf olle {ports} van {ypre} al tlant gheheel dat zi hadde ligghende onder mire joncvrauwe vorseit in de prochhie van {langhemarc} vte ghedaen een ymet soudlands dat blijft de vorseide {ucops} twelke land iloopt zesse ymeite litel min of litel meer En es ligghende in de sticke en in de pchelen hier naer genomt dats te weitene in een stic lants van zesse ymete ligghende tussche ser {jans} van der {meulin} lande an dene zide en de lan

In [4]:
print(ents, ents2)
len(ents), len(ents2)

['pauwels', 'vlaminc', 'fense', 'ghoedybeur', 'jan', 'wale', 'clais', 'hane', 'pasquedale', 'langhemarc', 'zulle', 'jacob', 'gherboud', 'yppre', 'janne', 'jans', 'clais', 'boitac', 'lisebette', 'keyen', 'willems', 'jakemine', 'ports', 'ypre', 'langhemarc', 'ucops', 'jans', 'meulin', 'frans', 'beisslare', 'Jte', 'fense', 'ghoedybeurs', 'adelijsse', 'ghilthuse', 'Jte', 'Jans', 'pauwels', 'hughemas', 'heinrijc', 'gheworpe', 'Jacob', 'gherboude', 'pasquendale', 'laumaent', 'vissensis'] ['pauwels', 'vlaminc', 'mich', 'de ghier', 'fense', 'ghoedybeur', 'jan', 'de wale', 'clais', 'de hane', 'pasquedale', 'langhemarc', 'jacob', 'gherboud', 'yppre', 'janne', 'de amman', 'jans', 'coste', 'clais', 'boitac', 'labrechtszone', 'lisebette', 'piet', 'keyen', 'willems', 'jakemine', 'ypre', 'langhemarc', 'jans', 'van der meulin', 'frans', 'van beisslare', 'fense', 'ghoedybeurs', 'adelijsse', 'van den ghilthuse', 'Jans', 'van der brigghe', 'pauwels', 'hughemas', 'heinrijc', 'sgroten', 'Jacob', 'gherboude

(46, 48)

In [20]:
# import spacy

# nlp = spacy.load("nl_core_news_sm")

In [22]:
# doc = nlp(txt)

# for ent in doc.ents:
#     print(ent)

zes
zes
drie
vier
akober gerboud
1300
twee
dertig
